In [ ]:
!pip install transformers


In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import io
from PIL import Image
import base64

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

image_paths = [
    'https://i.pinimg.com/originals/60/ad/86/60ad8627eb7eb880fcdd3b136b0239df.jpg',
    'https://elcyda.com/wp-content/uploads/2017/11/Geneva_SS_Water_Bottle_32oz_RED_MC0139_RD.jpg',
    'https://coolearthbottles.com/wp-content/uploads/2016/02/P1030117-769x1024.jpg',
    'https://www.cornellstore.com/site/Product_images/1598992_media-01.jpg',
    'https://n4.sdlcdn.com/imgs/a/3/c/Shopngift-Red-Metal-Water-Bottle-SDL243333394-1-77bb3.jpg',
    'https://www.highesthonor.biz/wp-content/uploads/2019/11/water-bottle-blue.jpg',
    'https://distributor.golding.eu/aluminium-water-bottle-400ml-cobalt-blue--7552-23--hd.jpg',
    'https://prizepossessions.com/wp-content/uploads/2020/01/8530.jpg'
]

descriptions = [
    'Red water bottle with black lid',
    'Red water bottle with silver lid',
    'Red water bottle with black and silver lid',
    'Red water bottle with white lid',
    'Red water bottle with plastic lid',
    'Blue water bottle with black lid',
    'Green water bottle with white lid',
    'Blue water bottle with silver lid'
]

color_categories = {
    "red": [],
    "blue": [],
    "green": [],

}

for img_path, description in zip(image_paths, descriptions):
    if "red" in description.lower():
        color_categories["red"].append(img_path)
    elif "blue" in description.lower():
        color_categories["blue"].append(img_path)
    elif "green" in description.lower():
        color_categories["green"].append(img_path)


color_labels = []
for description in descriptions:
    for color, images in color_categories.items():
        if any(color in description.lower() for color in color_categories.keys()):
            color_labels.append(color)
            break

data = {'Image': image_paths, 'Description': descriptions, 'Color': color_labels}
df = pd.DataFrame(data)

print(df)


                                               Image  \
0  https://i.pinimg.com/originals/60/ad/86/60ad86...   
1  https://elcyda.com/wp-content/uploads/2017/11/...   
2  https://coolearthbottles.com/wp-content/upload...   
3  https://www.cornellstore.com/site/Product_imag...   
4  https://n4.sdlcdn.com/imgs/a/3/c/Shopngift-Red...   
5  https://www.highesthonor.biz/wp-content/upload...   
6  https://distributor.golding.eu/aluminium-water...   
7  https://prizepossessions.com/wp-content/upload...   

                                  Description Color  
0             Red water bottle with black lid   red  
1            Red water bottle with silver lid   red  
2  Red water bottle with black and silver lid   red  
3             Red water bottle with white lid   red  
4           Red water bottle with plastic lid   red  
5            Blue water bottle with black lid   red  
6           Green water bottle with white lid   red  
7           Blue water bottle with silver lid   red  


In [ ]:
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")


In [ ]:
class Chatbot:
    def __init__(self, model, tokenizer, df):
        self.model = model
        self.tokenizer = tokenizer
        self.df = df

    def generate_response(self, prompt):
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
        outputs = self.model.generate(input_ids, max_length=1024)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

    def show_product_images(self, color):
        filtered_df = self.df[self.df['Color'] == color.lower()]
        if not filtered_df.empty:
            image_urls = filtered_df['Image'].tolist()
            return image_urls
        else:
            return []

    def describe_product_image(self, image_url):
        try:
            image_description = self.df[self.df['Image'] == image_url]['Description'].values[0]
            return image_description
        except Exception as e:
            print(e)
            return "I couldn't describe that image."

    def chat_with_user(self):
        while True:
            user_input = input("What would you like to do? ")

            if user_input == "show images":
                color = input("What color water bottle images would you like to see? ").lower()
                image_urls = self.show_product_images(color)
                if image_urls:
                    for url in image_urls:
                        print(f"Image URL: {url}")
                else:
                    print("No matching images found for the given color.")

            elif user_input == "describe image":
                image_url = input("Enter the URL of the image you would like to describe: ")
                image_description = self.describe_product_image(image_url)
                print(image_description)

            elif user_input == "exit":
                break

            else:
                response = self.generate_response(user_input)
                print(response)


In [ ]:
chatbot = Chatbot(model, tokenizer, df)

In [ ]:

chatbot.chat_with_user()

What would you like to do? show images
What color water bottle images would you like to see? red
Image URL: https://i.pinimg.com/originals/60/ad/86/60ad8627eb7eb880fcdd3b136b0239df.jpg
Image URL: https://elcyda.com/wp-content/uploads/2017/11/Geneva_SS_Water_Bottle_32oz_RED_MC0139_RD.jpg
Image URL: https://coolearthbottles.com/wp-content/uploads/2016/02/P1030117-769x1024.jpg
Image URL: https://www.cornellstore.com/site/Product_images/1598992_media-01.jpg
Image URL: https://n4.sdlcdn.com/imgs/a/3/c/Shopngift-Red-Metal-Water-Bottle-SDL243333394-1-77bb3.jpg
Image URL: https://www.highesthonor.biz/wp-content/uploads/2019/11/water-bottle-blue.jpg
Image URL: https://distributor.golding.eu/aluminium-water-bottle-400ml-cobalt-blue--7552-23--hd.jpg
Image URL: https://prizepossessions.com/wp-content/uploads/2020/01/8530.jpg
What would you like to do? describe image
Enter the URL of the image you would like to describe: https://elcyda.com/wp-content/uploads/2017/11/Geneva_SS_Water_Bottle_32oz_RED_